In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("data/gemstone.csv")

In [3]:
data = data.drop("id", axis=1)

In [4]:
X = data.drop("price", axis=1)

In [5]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [7]:
y = data.price

In [8]:
y

0         13619
1         13387
2          2772
3           666
4         14453
          ...  
193568     1130
193569     2874
193570     3036
193571      681
193572     2258
Name: price, Length: 193573, dtype: int64

In [11]:
categorical_cols = X.select_dtypes(include="object").columns

In [12]:
numerical_cols = X.select_dtypes(exclude="object").columns

In [13]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [14]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [16]:
num_pipeline = Pipeline(
    
    steps = [
        ("SimpleImputer", SimpleImputer()),
        ("StandardScaler", StandardScaler())
    ]
    
)

In [18]:
cat_pipeline = Pipeline(

    steps= [
        ("SimpleImputer", SimpleImputer(strategy='most_frequent')),
        ("OrdinalEncoder", OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories]))
    ]
)

In [19]:
preprocessor = ColumnTransformer(
        
    [
        ("numerical_pipeline" ,num_pipeline, numerical_cols),
        ("cat_pipeline",cat_pipeline, categorical_cols)
    ]
)

In [20]:
from sklearn.model_selection import train_test_split


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [22]:
preprocessor.fit_transform(X_train)

array([[ 2.62606128, -2.88812879,  0.40086761, ...,  3.        ,
         5.        ,  1.        ],
       [-0.84529098,  0.164716  ,  0.92245845, ...,  3.        ,
         1.        ,  2.        ],
       [-0.84529098, -1.50047207,  1.96564013, ...,  3.        ,
         2.        ,  2.        ],
       ...,
       [ 0.92272694,  0.90479959,  0.40086761, ...,  3.        ,
         3.        ,  3.        ],
       [-1.03934172, -0.66787803, -0.64231407, ...,  2.        ,
         3.        ,  6.        ],
       [-1.03934172, -0.02030489,  0.92245845, ...,  2.        ,
         3.        ,  1.        ]])

In [23]:
preprocessor.transform(X_test)

array([[-0.62967904,  0.25722645, -0.12072323, ...,  4.        ,
         0.        ,  2.        ],
       [ 2.60450008, -2.1480452 , -0.12072323, ...,  2.        ,
         3.        ,  1.        ],
       [-1.1255865 , -1.22294072,  0.92245845, ...,  3.        ,
         1.        ,  6.        ],
       ...,
       [-0.50031187, -0.48285713,  0.92245845, ...,  3.        ,
         1.        ,  3.        ],
       [-0.13377157,  0.44224735,  0.92245845, ...,  2.        ,
         5.        ,  1.        ],
       [ 0.51306425, -1.50047207, -0.64231407, ...,  4.        ,
         1.        ,  1.        ]])

In [24]:
preprocessor.get_feature_names_out()

array(['numerical_pipeline__carat', 'numerical_pipeline__depth',
       'numerical_pipeline__table', 'numerical_pipeline__x',
       'numerical_pipeline__y', 'numerical_pipeline__z',
       'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [32]:

X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [26]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
import numpy as np

In [27]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]



In [28]:
list(models)

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [29]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    print(model)

LinearRegression()
Lasso()
Ridge()
ElasticNet()


In [33]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1014.2749330568632
MAE: 674.735279609832
R2 score 93.63893549824442


Lasso
Model Training Performance
RMSE: 1014.3359560391641
MAE: 676.0356586934795
R2 score 93.63817005922814


Ridge
Model Training Performance
RMSE: 1014.2792090030064
MAE: 674.7697161746289
R2 score 93.63888186460788


Elasticnet
Model Training Performance
RMSE: 1514.3748258158537
MAE: 1051.464758663667
R2 score 85.81970787823009


